In [3]:
import os
import pickle
import scipy.stats as st
import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm

from funcs import compute_beta_vec
from funcs import get_optimal_betas_df
from beta_optimizer import BetaOptimizer
%load_ext autoreload
%autoreload 2

# Directory in which file with optimal betas are stored
generated_data_dir = 'generated_data'

# Constants
N = 100 # Number of cities
n_iterations = 10000 # Number of iterations for optimizing betas

In [4]:
class DatasetGenerator(object):
    def __init__(self, N=100):
        self.N = N
        self.x = None
        self.v = None
        self.refresh()

    def refresh(self):
        raise Exception("undefined")

class G1(DatasetGenerator):
    def refresh(self):
        self.x = st.uniform().rvs((self.N,2))
        self.v = st.uniform().rvs((self.N,))

class G2(DatasetGenerator):
    def refresh(self):
        self.x = st.uniform().rvs((self.N,2))
        self.v = np.exp(st.norm(-0.85, 1.3).rvs((self.N,)))

In [12]:
def compute_optimal_betas(model_string, N, lambda_vec, n_iterations):
    """Return a Pandas dataframe with optimal values of beta."""
    if model_string == 'G1':
        generator_class = G1
    else:
        generator_class = G2

    n_beta_iterations = 100
    epsilon_vec = np.linspace(0.05, 0.95, 10)
    beta_vec_list = []
    lambda_arrays_list = []
    for lambda_ in tqdm(lambda_vec):
        for i in range(n_beta_iterations):
            bo = BetaOptimizer(generator_class, lambda_, N, n_iterations, verbose=False)
            bo.run()
            try:
                beta_vec = compute_beta_vec(bo, epsilon_vec)
            except IndexError:
                pass
            else:
                beta_vec_list.append(beta_vec)
        lambda_arrays_list.append(np.array(beta_vec_list))

    optimal_betas_df = get_optimal_betas_df(lambda_vec, epsilon_vec, lambda_arrays_list, model_string)
    return optimal_betas_df

## Compute optimal $\beta(\epsilon)$

### Generative model $G_1$

In [8]:
lambda_vec = np.linspace(0, 1, 11)
optimal_betas_G1_df = compute_optimal_betas('G1', N, lambda_vec, n_iterations)

### Generative model $G_2$

In [13]:
lambda_vec = np.linspace(0, 2, 21)
optimal_betas_G2_df = compute_optimal_betas('G2', N, lambda_vec, n_iterations)

In [14]:
# Concatenate results for G1 and G2
optimal_betas_df = optimal_betas_G1_df.append(optimal_betas_G2_df)

# Save results to file
optimal_betas_df.to_csv(os.path.join(generated_data_dir, 'G1_and_G2_optimal_betas.csv'), index=False)